In [1]:
from dotenv import load_dotenv
import base64
import os
import http.client
import json

load_dotenv()

watsonx_api_key = os.getenv("WATSONX_APIKEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)
chat_url = os.getenv("IBM_WATSONX_AI_INFERENCE_URL", None)
### Encode image as base 64

pic = open("car.png","rb").read()
pic_base64 = base64.b64encode(pic)
pic_string = pic_base64.decode("utf-8")


In [2]:
conn_ibm_cloud_iam = http.client.HTTPSConnection(ibm_cloud_iam_url)
payload = "grant_type=urn%3Aibm%3Aparams%3Aoauth%3Agrant-type%3Aapikey&apikey="+watsonx_api_key
headers = { 'Content-Type': "application/x-www-form-urlencoded" }
conn_ibm_cloud_iam.request("POST", "/identity/token", payload, headers)
res = conn_ibm_cloud_iam.getresponse()
data = res.read()
decoded_json=json.loads(data.decode("utf-8"))
access_token=decoded_json["access_token"]

In [3]:
import requests
system_content = """You always answer the questions with markdown formatting using GitHub syntax. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.\n\nAny HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.\n\nWhen returning code blocks, specify language.\n\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. \nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information."""
user_message = """Please 1. Classify what object is this 2. Give a scale of 1-10 how damaged is the car 3. Do we need to change the parts or can fix. Please provide some descriptions\nAnswer in JSON with format {{'object': 'object_name', 'damage_scale': num, 'description': 'some_description'}}"""
body = {
   "messages": [
      {
         "role": "system",
         "content": system_content
      },
      {
         "role": "user",
         "content": [
            {
               "type": "text",
               "text": user_message,
            },
            {
               "type": "image_url",
               "image_url": {
                  "url": f"data:image/jpeg;base64, {pic_string}"
               }
            }
         ]
      }
   ],
   "project_id": project_id,
   # "model_id": "meta-llama/llama3-llava-next-8b-hf",
   # "model_id": "meta-llama/llama-3-2-11b-vision-instruct",
   "model_id": "meta-llama/llama-3-2-90b-vision-instruct",
   "decoding_method": "greedy",
   "repetition_penalty": 1.1,
   "max_tokens": 900
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"
}



response = requests.post(
    chat_url,
    headers=headers,
    json=body
)

if response.status_code != 200:
    raise Exception("Non-200 response: " + str(response.text))

data = response.json()

print('---- RESPONSE ----')
print(data['choices'][0]['message']['content'])

---- RESPONSE ----
```json
{
    "object": "car",
    "damage_scale": 6,
    "description": "The car has a noticeable dent on the rear bumper, but it does not appear to be severely damaged. The dent is likely to be repairable without requiring replacement of the entire bumper. However, a closer inspection by a professional mechanic or auto body specialist would be necessary to determine the extent of the damage and provide a more accurate assessment."
}
```
